In [276]:
import networkx as nx
import numpy as np
import cv2
import json
import networkx as nx
import re
import argparse
import glob
import os

from route_generator import iter_all_white_points, draw_waypoint, rdp_algorithm, recorded_find_path
from add_path_to_graph import add_path, read_adjlist_with_tuples

##################### Input Town number #####################

town = int(input("Enter town number: 1, 2, 3, 7, or 10"))

if town == 10:
    waypoint_map = cv2.imread("./waypoint_maps/Town10HD.png", 1)
    town_name = "Town10HD"
else:
    town_name = f"Town0{town}"
    waypoint_map = cv2.imread(f"./waypoint_maps/{town_name}.png", 1)


##################### Read latest graph list of the given town #####################

list_of_files = glob.glob(f'graph_list/{town_name}/*') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)

if town == 10:
    index = int(latest_file[30:-8])
    new_file_name = latest_file[:30] + str(index+1) + latest_file[-8:]
else:
    index = int(latest_file[27:-8])
    new_file_name = latest_file[:27] + str(index+1) + latest_file[-8:]

G = read_adjlist_with_tuples(latest_file)

print(latest_file)

graph_list/Town05\5_graph_v0.adjlist


In [273]:
################# Change start, end point #################

reverse = input("Do reverse search or not? (y for reverse search, other for original search)\n")

start, end = (2622, 853), (2593, 703)

if reverse == "y":
    start, end = end, start


In [274]:
if start == (0,0) or end == (0,0):
    print("Didn't change to your point!!!!!!!!")

all_whites_pos = iter_all_white_points(waypoint_map)

new_img = waypoint_map.copy()
new_img[start[0]][start[1]] = np.array([0, 0, 255])
new_img[end[0]][end[1]] = np.array([0, 0, 255])

cv2.circle(new_img, start[::-1], radius=5, color=(255, 0, 0), thickness=5)
cv2.circle(new_img, end[::-1], radius=5, color=(0, 255, 0), thickness=5)


print("Start point:", start)
print("End point:", end)
print("Finding Path...")
# final_path = find_path(start, end, all_whites_pos)
flag, final_path = recorded_find_path(start, end, all_whites_pos, "", False)

##### Path not found #####
if flag == False:
    print("Not found!!!")
    print(final_path)
    for i, j in final_path[0]:
        new_img[i][j] = np.array([255, 255, 0])
    
    draw_img = draw_waypoint(waypoint_map, start, end, final_path[0])
    rdp_img, _ = rdp_algorithm(draw_img, final_path[0])
    cv2.imwrite(f"test/not_found/test__{start}_{end}.png", new_img)
else:
    print("found!!!")
    print(final_path[0])
    print("Adding to the graph list...")
    
    if reverse == "y":
        fp = final_path[0][::-1]
        fp = [fp]
        add_path(G, fp, new_file_name)
    else:
        add_path(G, final_path, new_file_name)
    
    print(new_file_name, "Store!")
    draw_img = draw_waypoint(waypoint_map, start, end, final_path[0])
    rdp_img, _ = rdp_algorithm(draw_img, final_path[0])
    cv2.imwrite(f"test/found/test__{start}_{end}.png", rdp_img)

Start point: (4076, 3598)
End point: (4508, 3712)
Finding Path...
Get!!!
found!!!
[(4076, 3598), (4080, 3598), (4084, 3598), (4088, 3598), (4092, 3598), (4096, 3598), (4100, 3598), (4104, 3598), (4108, 3598), (4112, 3598), (4116, 3598), (4120, 3598), (4124, 3598), (4128, 3598), (4132, 3598), (4136, 3598), (4140, 3598), (4144, 3598), (4148, 3598), (4152, 3598), (4156, 3598), (4160, 3598), (4164, 3598), (4168, 3598), (4172, 3598), (4176, 3598), (4180, 3598), (4184, 3598), (4188, 3598), (4189, 3598), (4193, 3598), (4197, 3598), (4201, 3598), (4205, 3598), (4209, 3598), (4213, 3598), (4217, 3598), (4221, 3598), (4225, 3598), (4229, 3598), (4233, 3598), (4237, 3598), (4241, 3598), (4245, 3598), (4249, 3598), (4253, 3598), (4257, 3598), (4261, 3598), (4265, 3598), (4269, 3598), (4273, 3598), (4277, 3598), (4281, 3598), (4285, 3598), (4289, 3598), (4293, 3598), (4297, 3598), (4301, 3598), (4305, 3598), (4309, 3598), (4313, 3598), (4317, 3598), (4321, 3598), (4325, 3598), (4329, 3598), (4333, 

In [277]:
############### Quick test for other start point with the same goal point
start, end = (4508, 3712), (2593, 703)


fp = nx.shortest_path(G,  start,  end)
draw_img = draw_waypoint(waypoint_map, start, end, fp)
rdp_img, _ = rdp_algorithm(draw_img, fp)
cv2.imwrite(f"test/check__{start}_{end}.png", rdp_img)
print(fp)

[(4508, 3712), (4508, 3708), (4506, 3702), (4509, 3695), (4504, 3692), (4507, 3686), (4509, 3683), (4506, 3676), (4509, 3671), (4502, 3662), (4501, 3656), (4498, 3647), (4496, 3642), (4489, 3628), (4486, 3624), (4479, 3617), (4475, 3614), (4460, 3606), (4456, 3604), (4448, 3601), (4445, 3600), (4430, 3598), (4426, 3598), (4414, 3598), (4410, 3598), (4401, 3598), (4397, 3598), (4385, 3598), (4381, 3598), (4369, 3598), (4365, 3598), (4353, 3598), (4349, 3598), (4337, 3598), (4333, 3598), (4321, 3598), (4317, 3598), (4305, 3598), (4301, 3598), (4289, 3598), (4285, 3598), (4273, 3598), (4269, 3598), (4257, 3598), (4253, 3598), (4241, 3598), (4237, 3598), (4225, 3598), (4221, 3598), (4209, 3598), (4205, 3598), (4193, 3598), (4189, 3598), (4180, 3598), (4176, 3598), (4164, 3598), (4160, 3598), (4148, 3598), (4144, 3598), (4132, 3598), (4128, 3598), (4116, 3598), (4112, 3598), (4100, 3598), (4096, 3598), (4084, 3598), (4080, 3598), (4076, 3598)]
